# Imports

In [309]:
from nsepy import get_history
from datetime import date
import pandas as pd
import numpy as np
import talib
import datetime


# FETCHING DATA FROM NSE

In [248]:
scrip = 'HDFCAMC'

In [249]:

stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31)) 

In [250]:
pd.set_option('display.width', 10000)
print(f"Stock data size:{stock_df.shape}")
display(stock_df.head(2))
display(stock_df.tail(2))

Stock data size:(248, 14)


,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-01-01,HDFCAMC,EQ,2918.00,2937.65,3015.0,2935.0,3002.05,2998.05,2990.21,725339,2.168916e+14,49924,208435,0.2874
2021-01-04,HDFCAMC,EQ,2998.05,3015.95,3033.7,2970.4,3024.35,3025.65,3010.37,342044,1.029678e+14,28780,145996,0.4268


,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-12-30,HDFCAMC,EQ,2418.80,2420.90,2424.95,2382.0,2410.85,2408.55,2399.93,276518,6.636236e+13,18565,134253,0.4855
2021-12-31,HDFCAMC,EQ,2408.55,2415.95,2449.95,2411.0,2449.00,2446.10,2438.83,117665,2.869652e+13,12815,61668,0.5241


# STOCK ANALYSIS

## CANDLESTICK CHART

In [15]:
import plotly.graph_objects as go

candlestick = go.Candlestick(x=stock_df.index, 
                             open=stock_df['Open'],  
                             high=stock_df['High'], 
                             low=stock_df['Low'], 
                             close=stock_df['Close'])

fig = go.Figure(data=[candlestick])
fig.layout.xaxis.type = 'category' 
fig.show()

# CANDLESTICK PATTERNS

In [16]:
def plot_candle(df, title):
    candlestick = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])

    # Plot only the last month's data
    fig = go.Figure(data=[candlestick],
                    layout=go.Layout(title=go.layout.Title(text=title)))
    fig.layout.xaxis.type = 'category' 
    fig.show()


## MARUBOZU CANDLE - SINGLE CANDLESTICK PATTERN

In [17]:
# Identify the marubozu candles in the dataset
stock_df['marubozu'] = talib.CDLMARUBOZU(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the marubozu candles
marubozu_candles = stock_df[stock_df['marubozu'] != 0]
display(marubozu_candles[['Close','marubozu']])

# Plot the candlestick chart
plot_candle(marubozu_candles, title = f"Marubozu Candlestick Pattern - {scrip}")


,Close,marubozu
Date,,
2021-03-10,3118.50,-100
2021-03-18,2845.60,-100
2021-04-15,2949.50,100
2021-06-30,2917.45,-100
2021-10-18,2932.45,-100
2021-10-27,2701.25,-100
2021-12-06,2527.95,-100
2021-12-10,2549.75,-100
2021-12-13,2513.90,-100


## DOJI CANDLE

In [18]:
# Identify the doji candles in the dataset
stock_df['doji'] = talib.CDLDOJI(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the doji candles
doji_candles = stock_df[stock_df['doji'] != 0]
display(doji_candles[['Open','Close','doji']].tail())


# Plot the candlestick chart
plot_candle(doji_candles, title = f"Doji Candlestick Pattern - {scrip}")


,Open,Close,doji
Date,,,
2021-11-02,2660.4,2665.00,100
2021-11-04,2658.3,2654.95,100
2021-11-09,2670.0,2672.90,100
2021-11-15,2678.0,2680.75,100
2021-12-29,2415.0,2418.80,100


## ENGULFING CANDLE

In [23]:
# Identify the engulfing candles in the dataset
stock_df['engulf'] = talib.CDLENGULFING(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])

# Subset dataframe for only the doji candles
stock_df['next_is_engulf'] = stock_df['engulf'].shift(-1)
engulf_candles = stock_df[(stock_df['engulf'] != 0) | (stock_df['next_is_engulf'] != 0)]
display(engulf_candles[['Open','Close','engulf']].head(10))


# Plot the candlestick chart
plot_candle(engulf_candles, title = f"Engulfing Candlestick Pattern - {scrip}")


,Open,Close,engulf
Date,,,
2021-01-07,3119.75,3063.15,0
2021-01-08,3063.00,3137.85,100
2021-01-14,3259.00,3315.20,0
2021-01-15,3338.00,3244.40,-100
2021-01-28,2895.00,2920.85,0
2021-01-29,2938.10,2876.55,-100
2021-02-10,2998.65,2999.30,0
2021-02-11,3002.95,2979.40,-100
2021-03-01,2935.00,2938.75,0


## MORNING STAR/EVENING STAR CANDLE

In [28]:
stock_df[stock_df['eve_star'] != 0]

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,...,%Deliverble,marubozu,doji,engulf,next_is_engulf,morn_star,eve_star,star,next_1_star,next_2_star
Date,,,,,,,,,,,,,,,,,,,,,
2021-09-07,HDFCAMC,EQ,3188.75,3210.0,3210.0,3100.1,3129.9,3125.4,3145.32,266529,...,0.3858,0,0,0,0.0,0,-100,-100,0.0,0.0


In [69]:
# Identify the morning/evening candles in the dataset
stock_df['morn_star'] = talib.CDLMORNINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])           
stock_df['eve_star']  = talib.CDLEVENINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
stock_df['star'] = stock_df['morn_star'] + stock_df['eve_star'] 
# Subset dataframe for only the doji candles
stock_df['next_1_star'] = stock_df['star'].shift(-1, fill_value=0)
stock_df['next_2_star'] = stock_df['star'].shift(-2, fill_value=0)
star_candles = stock_df[(stock_df['star'] != 0) | (stock_df['next_1_star'] != 0) | (stock_df['next_2_star'] != 0)]
display(star_candles[['Close','morn_star','eve_star']].head(10))


# Plot the candlestick chart
plot_candle(star_candles, title = f"Morning/Evening Candlestick Pattern - {scrip}")


,Close,morn_star,eve_star
Date,,,
2021-09-03,3164.15,0,0
2021-09-06,3188.75,0,0
2021-09-07,3125.40,0,-100


# TRADING STRATEGIES/INDICATORS

## MOVING AVERAGE

### SIMPLE MOVING AVERAGE

In [72]:
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df[['Close','20_sma']].tail()

,Close,20_sma
Date,,
2021-12-27,2372.20,2466.8525
2021-12-28,2417.90,2462.1825
2021-12-29,2418.80,2455.5625
2021-12-30,2408.55,2448.1550
2021-12-31,2446.10,2441.4925


In [73]:
candlestick1 = go.Candlestick(x=stock_df.index, open=stock_df['Open'], high=stock_df['High'], low=stock_df['Low'], close=stock_df['Close'])
candlestick1





Candlestick({
    'close': array([2998.05, 3025.65, 3091.95, ..., 2418.8 , 2408.55, 2446.1 ]),
    'high': array([3015.  , 3033.7 , 3120.  , ..., 2433.95, 2424.95, 2449.95]),
    'low': array([2935.  , 2970.4 , 2987.95, ..., 2406.05, 2382.  , 2411.  ]),
    'open': array([2937.65, 3015.95, 3000.  , ..., 2415.  , 2420.9 , 2415.95]),
    'x': array([datetime.date(2021, 1, 1), datetime.date(2021, 1, 4),
                datetime.date(2021, 1, 5), ..., datetime.date(2021, 12, 29),
                datetime.date(2021, 12, 30), datetime.date(2021, 12, 31)], dtype=object)
})

### MOVING AVERAGE CHART

In [74]:
# Compute the moving averages for 20 days, 50 days and 200 days
stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

# Plot the moving averages along with the candlestick chart
small_df = stock_df.tail(40)
candlestick = go.Candlestick(x=small_df.index, open=small_df['Open'], high=small_df['High'], low=small_df['Low'], close=small_df['Close'])
sma_20 = go.Scatter(x=small_df.index, y=small_df['20_sma'], name='20 days MA', line={'color': 'blue'})
sma_50 = go.Scatter(x=small_df.index, y=small_df['50_sma'], name='50 days MA', line={'color': 'orange'})
sma_200 = go.Scatter(x=small_df.index, y=small_df['200_sma'], name='200 days MA', line={'color': 'brown'})

# Plot only the last month's data
fig = go.Figure(data=[candlestick,sma_20,sma_50,sma_200],
                layout=go.Layout(title=go.layout.Title(text=f"MOVING AVERAGE CHART - {scrip}")))
fig.layout.xaxis.type = 'category' 
fig.show()


## RSI

In [83]:
import talib
RSI_PERIOD = 14
stock_df['rsi'] = talib.RSI(stock_df['Close'], RSI_PERIOD) 
stock_df['rsi_indicator'] = stock_df['rsi'].apply(lambda x: 'BUY' if x<30 else 'SELL' if x>70 else 'HOLD')
display(stock_df[['Close','rsi','rsi_indicator']].tail(15))

,Close,rsi,rsi_indicator
Date,,,
2021-12-13,2513.90,36.900579,HOLD
2021-12-14,2487.20,34.181544,HOLD
2021-12-15,2456.75,31.344874,HOLD
2021-12-16,2437.75,29.689229,BUY
2021-12-17,2397.95,26.528239,BUY
2021-12-20,2313.60,21.342062,BUY
2021-12-21,2327.15,23.915272,BUY
2021-12-22,2357.55,29.488558,BUY
2021-12-23,2378.30,33.091240,HOLD


## MACD

In [206]:
scrip = 'HINDUNILVR'
stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [230]:
from plotly.subplots import make_subplots

def plot_macd(stock_df, macd, signal, hist):
    fig = make_subplots( rows=2, cols=1)

    candlestick = go.Candlestick(x=stock_df.index, 
                                 open=stock_df['Open'],  
                                 high=stock_df['High'], 
                                 low=stock_df['Low'], 
                                 close=stock_df['Close'], name = 'daily candle')

    positive_hist = hist[hist>0]
    negative_hist = hist[hist<0]

    macd_line = go.Scatter(x=macd.index, y = macd, name = 'macd',line_color = 'blue')
    signal_line = go.Scatter(x=signal.index, y = signal, name = 'signal',line_color = 'orange')
    pos_hist_bar = go.Bar(x=positive_hist.index, y = positive_hist, name = 'hist',marker={'color': 'green'})
    neg_hist_bar = go.Bar(x=negative_hist.index, y = negative_hist, name = 'hist',marker={'color': 'red'})

    fig.add_trace(candlestick, row=1,col=1)    
    fig.add_trace(macd_line, row=2, col=1)
    fig.add_trace(signal_line, row=2, col=1)
    fig.add_trace(pos_hist_bar, row=2, col=1)
    fig.add_trace(neg_hist_bar, row=2, col=1)

    fig.update_layout(title_text=f'MACD - {scrip}', title_x=0.5,
                     xaxis_type = 'category',xaxis_rangeslider_visible = False,
                     xaxis_showticklabels = False,
                     xaxis2_type = 'category' )
    
    fig.show()


In [231]:
import talib

MACD_FAST_EMA = 12
MACD_SLOW_EMA = 26
MACD_SIGNAL_PERIOD = 9 

macd, signal, hist = talib.MACD(stock_df['Close'], 
                                fastperiod=MACD_FAST_EMA, slowperiod=MACD_SLOW_EMA, signalperiod=MACD_SIGNAL_PERIOD)

plot_macd(stock_df, macd, signal, hist)

## BOLLINGER BAND

In [290]:
scrip = 'ITC'
stock_df = get_history(symbol=scrip,
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [298]:
# 20-day moving average 
stock_df['sma_bb'] = stock_df['Close'].rolling(window=20).mean()

# upper and lower bollinger bands: SMA +/- 2 * standard deviation
stock_df['stddev'] = stock_df['Close'].rolling(window=20).std()
stock_df['upper_bb'] = stock_df['sma_bb'] + (2 * stock_df['stddev'])
stock_df['lower_bb'] = stock_df['sma_bb'] - (2 * stock_df['stddev'])

display(stock_df[['Close','sma_bb','upper_bb','lower_bb']]).tail()


# TALib - Bollinger Band
# upperband, middleband, lowerband = BBANDS(stock_df['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)


,Close,sma_bb,upper_bb,lower_bb
Date,,,,
2021-01-01,213.85,NaN,NaN,NaN
2021-01-04,213.40,NaN,NaN,NaN
2021-01-05,211.45,NaN,NaN,NaN
2021-01-06,205.40,NaN,NaN,NaN
2021-01-07,202.80,NaN,NaN,NaN
2021-01-08,201.50,NaN,NaN,NaN
2021-01-11,202.50,NaN,NaN,NaN
2021-01-12,206.45,NaN,NaN,NaN
2021-01-13,211.25,NaN,NaN,NaN


AttributeError: 'NoneType' object has no attribute 'tail'

In [299]:
def plot_bollinger_band(stock_df):

    candlestick = go.Candlestick(x=stock_df.index, 
                                 open=stock_df['Open'],  
                                 high=stock_df['High'], 
                                 low=stock_df['Low'], 
                                 close=stock_df['Close'], name = 'Daily candle')


    upper_line = go.Scatter(x=stock_df.index, y = stock_df['upper_bb'], name='upper_band',line_color = 'blue')
    lower_line = go.Scatter(x=stock_df.index, y = stock_df['lower_bb'], name = 'lower_band',line_color = 'blue')
    sma_line = go.Scatter(x=stock_df.index, y = stock_df['sma_bb'], name = 'sma_line',line_color = 'lightblue')   

                                                         
    fig = go.Figure(data=[candlestick, upper_line, lower_line, sma_line])
                                                         
    fig.update_layout(title_text=f'Bollinger Band - {scrip}', title_x=0.5,
                     xaxis_type = 'category',xaxis_rangeslider_visible = False,
                     xaxis_showticklabels = True )
    
    fig.show()
    
plot_bollinger_band(stock_df)

# BUILDING A SIMPLE TRADING STRATEGY

TRADING STRATEGY:  
1. Get the individual BUY/SELL/HOLD indicators based on the following strategies: Moving Average, MACD, Engulfing Pattern  
2. If even one of the strategies provide a BUY/SELL recommendation, then check the RSI value
3. If more than one indicator gives a signal, choose in this order: MACD, Moving Average, Engulfing Pattern  
4. RSI value of last 3 sessions considered
5. If strategy indicates a BUY and RSI < lower_threshold (40) --> Final Indicator = 'BUY'  
6. If strategy indicates a BUY and RSI > upper_threshold (60) --> Final Indicator = 'SELL' 


In [609]:
# Get additional data to calculate the 200-d moving average from 1st Jan
start_date = date(2021,1,1) - datetime.timedelta(300) 
def fetch_data(scrip_name):
    return get_history(symbol=scrip_name,
                    start=start_date, 
                    end=date(2021,12,31))

In [610]:
# Compute RSI
def apply_rsi_strategy(stock_df, rsi_period = 14):
    stock_df['rsi'] = talib.RSI(stock_df['Close'], rsi_period) 
    return stock_df

In [611]:
# BUY/SELL/HOLD indicator based on the 200-days, 50-days and 20-days simple moving average
def get_ma_indicator(row):
    indicator = 'HOLD'
    row_values = [row['Close'], row['20_sma'], row['50_sma'] , row['200_sma']]
    
    # If close < 20-day sma < 50-day sma < 200-day sma --> BUY
    # If close > 20-day sma > 50-day sma > 200-day sma --> SELL
    if(row_values == sorted(row_values)):
        indicator = 'BUY'
    elif (row_values == sorted(row_values, reverse=True)):
        indicator = 'SELL'
        
    return indicator

def apply_moving_average_strategy(stock_df):
    # Compute the moving averages for 20 days, 50 days and 200 days
    stock_df['20_sma'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_sma'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['200_sma'] = stock_df['Close'].rolling(window=200).mean()

    stock_df['MA_indicator'] = stock_df.apply(lambda row: get_ma_indicator(row), axis=1)
    return stock_df
        

In [612]:
# BUY/SELL/HOLD indicator based on the macd crossover

def get_macd_indicator(row):
    last_hist = row['hist']
    prev_hist = row['prev_hist']
    indicator = 'HOLD'
    # Provide BUY or SELL indications during crossovers
    if not np.isnan(prev_hist) and not np.isnan(last_hist):
        # If hist value has changed from negative to positive or vice versa, it indicates a crossover
        macd_crossover = (abs(last_hist + prev_hist)) != (abs(last_hist) + abs(prev_hist))
        if macd_crossover:
            indicator = 'BUY' if last_hist > 0 else 'SELL'
            
    return indicator
        
def apply_macd_strategy(stock_df):
    stock_df['macd'],  stock_df['signal'],  stock_df['hist'] = talib.MACD(stock_df['Close'], 
                                fastperiod=12, slowperiod=26, signalperiod=9)
    stock_df['prev_hist'] = stock_df['hist'].shift(1)
    stock_df['macd_indicator'] = stock_df.apply(lambda row: get_macd_indicator(row), axis=1)
    
    return stock_df
    

In [613]:
# BUY/SELL/HOLD indicator based on the engulfing pattern

def apply_engulfing_pattern(stock_df):
    stock_df['engulf'] = talib.CDLENGULFING(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
    stock_df['engulf_indicator'] = stock_df.apply(lambda row: 'BUY' if (row['engulf'] > 0) else 'SELL' if (row['engulf'] < 0) else 'HOLD', axis=1)
    return stock_df



In [614]:
CHECK_RSI_LAST_SESSIONS = 3
RSI_LOWER_LIMIT = 40
RSI_UPPER_LIMIT = 60
def apply_trading_algo(stock_df):
    stock_df['indicator'] = 'HOLD'
    buy_sell_signals = stock_df[(stock_df['macd_indicator'] != 'HOLD') | (stock_df['MA_indicator'] != 'HOLD')| (stock_df['engulf_indicator'] != 'HOLD')]
    for index,row in buy_sell_signals.iterrows():
        # Get previous n sessions whenever a signal is reached 
        analysis_df = stock_df.loc[:index].tail(3)
        macd_ind = analysis_df.iloc[-1]['macd_indicator']
        ma_ind = analysis_df.iloc[-1]['MA_indicator']
        engulf_ind = analysis_df.iloc[-1]['engulf_indicator']
        current_indicator = 'HOLD' 
        
        # Preference given in this order: MACD, MA, engulfing pattern
        current_indicator = macd_ind if macd_ind != 'HOLD' else \
                            ma_ind if ma_ind != 'HOLD' else engulf_ind
        
        # Decide the final indicator based on the RSI value
        if ((current_indicator == 'BUY') and (analysis_df['rsi'].min() <= RSI_LOWER_LIMIT)) \
            or \
            ((current_indicator == 'SELL') and (analysis_df['rsi'].max() >= RSI_UPPER_LIMIT)): 
            stock_df.at[index, 'indicator'] = current_indicator
        
    return stock_df



,Symbol,Close,indicator,MA_indicator,rsi,rsi_indicator,macd_indicator,engulf_indicator
Date,,,,,,,,
2021-01-01,ITC,213.85,SELL,SELL,64.541674,HOLD,HOLD,HOLD
2021-01-04,ITC,213.40,SELL,SELL,63.659955,HOLD,HOLD,HOLD
2021-01-05,ITC,211.45,SELL,SELL,59.844711,HOLD,HOLD,HOLD
2021-01-14,ITC,214.15,SELL,SELL,61.627592,HOLD,HOLD,HOLD
2021-01-15,ITC,217.85,SELL,SELL,65.306936,HOLD,HOLD,HOLD
2021-01-19,ITC,218.85,SELL,SELL,65.374618,HOLD,HOLD,HOLD
2021-01-20,ITC,217.65,SELL,SELL,63.105293,HOLD,HOLD,HOLD
2021-01-21,ITC,213.70,SELL,SELL,56.190902,HOLD,HOLD,HOLD
2021-01-22,ITC,210.70,SELL,HOLD,51.569342,HOLD,SELL,HOLD


In [615]:
def plot_buy_sell_chart(scrip_name, stock):
    candlestick = go.Candlestick(x=stock.index, 
                                 open=stock['Open'],  
                                 high=stock['High'], 
                                 low=stock['Low'], 
                                 close=stock['Close'])

    fig = go.Figure(data=[candlestick],
                   layout=go.Layout(title=go.layout.Title(text=f"TRADING ALGO - BUY/SELL RECOMMENDATIONS - {scrip_name}")))
    fig.layout.xaxis.type = 'category' 
    fig.layout.xaxis.rangeslider.visible = False

    for index, row in stock.iterrows(): 
        #print(row)
        if (row['indicator'] != 'HOLD'):
            line_colour = 'orange' if (row['indicator'] == 'BUY') else 'purple'
            fig.add_vline(x=row.name, line_width=3, line_dash="dash", line_color=line_colour)

            
    fig.show()
    
    

In [616]:
def my_trading_algo(scrip_name):
    # Fetch the NSE data
    stock = fetch_data(scrip_name)

    # Apply the strategies individual
    stock = apply_moving_average_strategy(stock)
    stock = apply_rsi_strategy(stock)
    stock = apply_macd_strategy(stock)
    stock = apply_engulfing_pattern(stock)

    # Get only 2021 data for further analysis
    stock = stock[stock.index >= date(2021,1,1)]

    # Merge the strategies to create a final BUY/SELL/HOLD indicator
    stock = apply_trading_algo(stock)

    # Plot the buy/sell recommendations on 2021 data
    plot_buy_sell_chart(scrip_name, stock)
    
    return stock

In [617]:
hul = my_trading_algo('HINDUNILVR')

In [618]:
itc = my_trading_algo('ITC')

In [619]:
hdfcamc = my_trading_algo('HDFCAMC')

In [620]:
whirlpool = my_trading_algo('WHIRLPOOL')

In [621]:
infy = my_trading_algo('INFY') # RSI is never less than 40 for INFY - hence no BUY calls

In [622]:
#  FOR DEBUGGING
infy[['Symbol','Close','indicator','rsi','MA_indicator','macd_indicator','engulf_indicator']]

,Symbol,Close,indicator,rsi,MA_indicator,macd_indicator,engulf_indicator
Date,,,,,,,
2021-01-01,INFY,1260.45,SELL,70.464794,SELL,HOLD,HOLD
2021-01-04,INFY,1288.25,SELL,74.540831,SELL,HOLD,HOLD
2021-01-05,INFY,1293.80,SELL,75.274460,SELL,HOLD,HOLD
2021-01-06,INFY,1282.10,SELL,70.652386,SELL,HOLD,HOLD
2021-01-07,INFY,1262.15,SELL,63.493282,SELL,HOLD,HOLD
2021-01-08,INFY,1312.10,SELL,71.327203,SELL,HOLD,HOLD
2021-01-11,INFY,1376.20,SELL,77.885496,SELL,HOLD,HOLD
2021-01-12,INFY,1371.75,SELL,76.576012,SELL,HOLD,HOLD
2021-01-13,INFY,1387.15,SELL,77.957209,SELL,HOLD,HOLD


# OTHERS

## MORNING & EVENING STAR

In [39]:

stock_df = get_history(symbol='DMART',
                    start=date(2021,1,1), 
                    end=date(2021,12,31))

In [46]:
# Identify the morning/evening candles in the dataset
stock_df['morn_star'] = talib.CDLMORNINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])           
stock_df['eve_star']  = talib.CDLEVENINGSTAR(stock_df['Open'], stock_df['High'], stock_df['Low'], stock_df['Close'])
stock_df['star'] = stock_df['morn_star'] + stock_df['eve_star'] 
# Subset dataframe for only the doji candles
stock_df['next_1_star'] = stock_df['star'].shift(-1, fill_value=0)
stock_df['next_2_star'] = stock_df['star'].shift(-2, fill_value=0)
star_candles = stock_df[(stock_df['star'] != 0) | (stock_df['next_1_star'] != 0) | (stock_df['next_2_star'] != 0)]
display(star_candles[['Close','morn_star','eve_star']])


# Plot the candlestick chart
plot_candle(star_candles, title = f"Morning/Evening Candlestick Pattern - DMART")


,Close,morn_star,eve_star
Date,,,
2021-04-22,2717.60,0,0
2021-04-23,2713.70,0,0
2021-04-26,2808.25,100,0
2021-08-31,3954.80,0,0
2021-09-01,3968.60,0,0
2021-09-02,3917.65,0,-100


In [66]:
plot_candle(stock_df.loc[date(2021,3,26):date(2021,5,10)], title = f"DMART - MORNING CANDLE")
plot_candle(stock_df.loc[date(2021,8,10):date(2021,9,15)], title = f"DMART - EVENING CANDLE")